What is Vulngpt ?
-This is 

**Import the Necessary libraries **

In [2]:
!pip install transformers pandas torch flask beautifulsoup4 requests


In [1]:
import requests
from bs4 import BeautifulSoup
import re

def scrape_website(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.prettify()
    except requests.exceptions.RequestException as e:
        print(f"Error while fetching the URL: {e}")
        return None

def extract_sql_queries(code):
    # Regex to find SQL queries (basic example)
    sql_pattern = re.compile(r"(SELECT|INSERT|UPDATE|DELETE)\s.*\s(FROM|INTO|SET)\s.*", re.IGNORECASE)
    queries = sql_pattern.findall(code)
    return [' '.join(query) for query in queries]

def detect_sqli(query):
    # Simple rule-based detection (for illustration purposes)
    sqli_patterns = [
        r"'.*'",               # Unescaped single quotes
        r"\".*\"",             # Unescaped double quotes
        r"OR\s1=1",            # Typical SQLi payload
        r"--",                 # Comment injection
        r";\sDROP\sTABLE",     # Dangerous SQL keywords
        
    ]

    for pattern in sqli_patterns:
        if re.search(pattern, query, re.IGNORECASE):
            return True
    return False


url = 'https://www.facebook.com/'  # Replace with the URL you want to test
website_code = scrape_website(url)

if website_code:
    # Extract SQL queries from the website code
    sql_queries = extract_sql_queries(website_code)
    print(f"SQL Queries Found: {sql_queries}")  # Debugging line to check the content

    if sql_queries:
        # Detect SQLi in each SQL query
        for query in sql_queries:
            if detect_sqli(query):
                print(f"Possible SQL Injection vulnerability detected in query: {query}")
            else:
                print(f"Safe SQL query: {query}")
    else:
        print("No SQL queries found.")
else:
    print("Failed to retrieve website code.")


SQL Queries Found: ['Delete from']
Safe SQL query: Delete from


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# Load pre-trained RoBERTa tokenizer and model
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)  

# Define vulnerability types that RoBERTa can detect
vulnerability_types = {
    0: 'Safe',
    1: 'Cross-Site Scripting (XSS)',
    2: 'Cross-Site Request Forgery (CSRF)',
    3: 'Remote Code Execution (RCE)',
}
# Detect vulnerabilities using RoBERTa
def detect_vulnerabilities(code):
    inputs = roberta_tokenizer(code, return_tensors='pt', max_length=512, truncation=True)
    outputs = roberta_model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return vulnerability_types[prediction]
# Example usage with website code
vulnerability_status = detect_vulnerabilities(website_code)
print(f"Vulnerability Detected: {vulnerability_status}")

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load GPT-2 tokenizer and model
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

# Generate remediation suggestions
def generate_remediation_suggestion(vulnerability_type):
    input_text = f"The following vulnerability was detected: {vulnerability_type}. Suggested remediation: "
    input_ids = gpt2_tokenizer.encode(input_text, return_tensors='pt')
    output = gpt2_model.generate(input_ids, max_length=100, num_return_sequences=1)
    suggestion = gpt2_tokenizer.decode(output[0], skip_special_tokens=True)
    return suggestion

# Example usage if the website is vulnerable
if vulnerability_status != 'Safe':
    remediation_suggestion = generate_remediation_suggestion(vulnerability_status)
    print(f"Remediation Suggestion: {remediation_suggestion}")
else:
    print("No vulnerabilities detected.")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

NameError: name 'vulnerability_status' is not defined

**Dataset Preparing :**

In [5]:
!pip install pandas


In [6]:
import pandas as pd

# Load dataset
dataset_path = '/kaggle/input/vulnerbility/vulnerability_dataset.csv'  # Path to your dataset
data = pd.read_csv(dataset_path)

# Preview the dataset
print(data.head())


                                     code          label
0          <script>alert('XSS');</script>            XSS
1      SELECT * FROM users WHERE id = $id  SQL Injection
2   <form action='/submit' method='POST'>           CSRF
3                  <h1>Hello, World!</h1>           Safe
4  <img src='invalid' onerror='alert(1)'>            XSS


In [7]:
from transformers import RobertaTokenizer
from sklearn.model_selection import train_test_split

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example['code'], padding="max_length", truncation=True, max_length=512)

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data['code'], data['label'], test_size=0.2)

# Tokenize the training and test data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [11]:
from sklearn.preprocessing import LabelEncoder

# Convert string labels into integers
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)


In [14]:
import torch
from torch.utils.data import Dataset

class VulnerabilityDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create PyTorch dataset objects
train_dataset = VulnerabilityDataset(train_encodings, train_labels)
test_dataset = VulnerabilityDataset(test_encodings, test_labels)


In [15]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained RoBERTa model with classification head
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoder.classes_))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="steps"
)

# Create Trainer instance
trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

# Fine-tune the model
trainer.train()


2024-09-06 07:03:00.498190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-06 07:03:00.498331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 07:03:00.619390: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Pa

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
10,2.359900,2.353521
20,2.316200,2.318360
30,2.281800,2.251393
40,2.167500,2.088924
50,1.879700,1.262213
60,1.120500,0.882433
70,0.910600,0.670563
80,0.788800,0.541834
90,0.526500,0.481046
100,0.341500,0.423522


TrainOutput(global_step=222, training_loss=0.8383341494712744, metrics={'train_runtime': 471.0578, 'train_samples_per_second': 3.732, 'train_steps_per_second': 0.471, 'total_flos': 29814885095688.0, 'train_loss': 0.8383341494712744, 'epoch': 3.0})

In [39]:
# Load the fine-tuned model and tokenizer
model = RobertaForSequenceClassification.from_pretrained('./fine_tuned_roberta')
tokenizer = RobertaTokenizer.from_pretrained('./fine_tuned_roberta')

# Detect vulnerabilities in the website code
def detect_vulnerability_with_fine_tuned_model(code):
    inputs = tokenizer(code, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([prediction])[0]


vulnerability_type = detect_vulnerability_with_fine_tuned_model(website_code)
print(f"Detected vulnerability: {vulnerability_type}")


OSError: Incorrect path_or_model_id: './fine_tuned_roberta'. Please provide either the path to a local folder or the repo_id of a model on the Hub.